## Imports

In [1]:
import pandas as pd
import numpy as np
import os
import folium
import ast
import textwrap

## Data Read in

In [2]:
df = pd.read_csv("Foiled Office Projects in South Florida  - USE THIS TAB.csv")

## Map Work

In [3]:
df.columns

Index(['Project ', 'Coordinates ', 'Caption'], dtype='object')

In [4]:
df['Coordinates '].dtype

dtype('O')

In [5]:
df.head(2)

,Project,Coordinates,Caption
0,Swire's One Brickell City Centre,"25.766976993450072, -80.19107448540062",Swire nixed the planned supertall at Brickell ...
1,Gateway Group's Mid-Beach offices,"25.812815434081426, -80.13546505236465",Although the Giller family's Gateway Group had...


In [6]:
import ast, textwrap, pandas as pd, folium
from folium.plugins import MarkerCluster, Fullscreen, MiniMap

def tooltip_html(df, row_index):
    project = df.at[row_index, 'Project ']
    caption = df.at[row_index, 'Caption']
    project_str = str(project).strip() if pd.notnull(project) else 'N/A'
    caption_str = str(caption).strip() if pd.notnull(caption) else 'N/A'
    
    tooltip_content = f"""
    <div class="popup-content">
        <strong>Project:</strong> {project_str}<br>
        <p>{caption_str}</p>
    </div>
    """
    return tooltip_content

# Assuming df is already defined and loaded with your data
# Initialize the map centered around the first geocoded point
lat_str, lon_str = df['Coordinates '].iloc[0].split(',')
map_center = [float(lat_str.strip()), float(lon_str.strip())]
m = folium.Map(location=map_center, zoom_start=12, scrollWheelZoom=False)

# Add custom Mapbox tile layer
folium.TileLayer(
    tiles='https://api.mapbox.com/styles/v1/mapbox/streets-v11/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ',
    attr='Mapbox',
    name='Streets',
    overlay=True,
    control=False,
    show=False,
    min_zoom=1,
    max_zoom=20
).add_to(m)

# Add custom CSS to style the tooltips and popups
custom_css = """
<style>
    .popup-content {
        min-width: 300px;  /* Ensures the popup is at least 300px wide */
        font-size: 14px;
        line-height: 1.4;
        color: #333;
        white-space: normal;  /* Allow natural wrapping */
        word-wrap: break-word;
    }
    .leaflet-popup, .leaflet-popup-content-wrapper {
        background-color: #f9f9f9;
        border: 1px solid #bbb;
        border-radius: 5px;
        padding: 8px;
        box-shadow: 0 2px 6px rgba(0,0,0,0.1);
    }
    .leaflet-popup-tip {
        background: #f9f9f9;
    }
</style>

"""
m.get_root().html.add_child(folium.Element(custom_css))

# Add title to the map
title_html = '''
    <h3 style="text-align:center; font-family:Arial, sans-serif; font-size:18px; color:#333; margin-top:10px;">
        <b>Foiled Office Projects</b>
    </h3>
'''
m.get_root().html.add_child(folium.Element(title_html))

# Add additional controls
Fullscreen().add_to(m)
MiniMap(toggle_display=True).add_to(m)

# Create marker cluster
marker_cluster = MarkerCluster().add_to(m)

# Loop through the DataFrame to create markers
for i, row in df.iterrows():
    lat_str, lon_str = row['Coordinates '].split(',')
    lat, lon = float(lat_str.strip()), float(lon_str.strip())
    tooltip_content = tooltip_html(df, i)
    
    # Using a Marker with an icon as an example (you can switch back to CircleMarker if preferred)
    folium.Marker(
        location=[lat, lon],
        popup=folium.Popup(tooltip_content, max_width=300),
        icon=folium.Icon(color='red', icon='info-sign')
    ).add_to(m)

# Display the map
m


In [7]:
m.save('index.html')

In [8]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/FoiledOfficeProjectsInSouthFlorida
